In [1]:
import cdsapi
import xarray as xr
import rioxarray
import configparser as cfg
import yaml
from tqdm import tqdm
from atmospheric_explorer.cams_interface.ghg import InversionOptimisedGreenhouseGas

In [2]:
def add_var(data, var):
    df = xr.open_dataset('download.nc')
    data_vars_list = [v for v in df.data_vars.keys()]
    data_vars = ",".join(data_vars_list)
    units = ",".join([df[v].attrs['units'] for v in data_vars_list])
    long_names = ",".join([df[v].attrs['long_name'] for v in data_vars_list])
    data[var] = {
        "factor": 1,
        "original_units": units,
        "convert_units": units,
        "cams_variables": data_vars,
        "long_names": long_names
    }

In [3]:
data = {}
skipped = []
c = cdsapi.Client()

In [4]:
c.retrieve(
    'cams-global-greenhouse-gas-inversion',
    {
        'version': 'latest',
        'format': 'zip',
        'variable': 'carbon_dioxide',
        'quantity': 'mean_column',
        'input_observations': 'surface',
        'year': '1979',
        'month': '01',
        'time_aggregation': 'instantaneous',
    },
    'download.zip')

2023-07-15 15:51:31,402 INFO Welcome to the CDS
2023-07-15 15:51:31,403 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-global-greenhouse-gas-inversion
2023-07-15 15:51:31,439 INFO Request is queued
2023-07-15 15:51:32,465 INFO Request is running
2023-07-15 15:51:34,011 INFO Request is completed
2023-07-15 15:51:34,012 INFO Downloading https://download-0000-ads-clone.copernicus-climate.eu/cache-compute-0000/cache/data1/dataset-cams-global-greenhouse-gas-inversion-2d155517-7d5d-4a4c-9e66-bd11694b85e1.zip to download.zip (5.8M)
2023-07-15 15:51:36,964 INFO Download rate 2M/s     


Result(content_length=6060706,content_type=application/zip,location=https://download-0000-ads-clone.copernicus-climate.eu/cache-compute-0000/cache/data1/dataset-cams-global-greenhouse-gas-inversion-2d155517-7d5d-4a4c-9e66-bd11694b85e1.zip)

In [5]:
df = InversionOptimisedGreenhouseGas(
    data_variables='carbon_dioxide',
    file_format='zip',
    quantity='mean_column',
    input_observations='surface',
    time_aggregation='instantaneous',
    year='1990',
    month=['01', '02', '03']
)
df.download()

2023-07-15 15:51:36,977 INFO Created folder /home/luigibrancati/.atmospheric_explorer/data/global_greenhouse_gas_inversion
2023-07-15 15:51:36,979 INFO Created folder /home/luigibrancati/.atmospheric_explorer/data/global_greenhouse_gas_inversion/data_2
2023-07-15 15:51:37,004 INFO Welcome to the CDS
2023-07-15 15:51:37,005 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-global-greenhouse-gas-inversion
2023-07-15 15:51:37,038 INFO Request is queued
2023-07-15 15:51:38,107 INFO Request is running
2023-07-15 15:51:39,643 INFO Request is completed
2023-07-15 15:51:39,644 INFO Downloading https://download-0000-ads-clone.copernicus-climate.eu/cache-compute-0000/cache/data1/dataset-cams-global-greenhouse-gas-inversion-9fb21dee-6c3b-419c-ac64-4068749f39e2.zip to /home/luigibrancati/.atmospheric_explorer/data/global_greenhouse_gas_inversion/data_2/data_2.zip (16.7M)
2023-07-15 15:51:49,958 INFO Download rate 1.6M/s   
2023-07-15 15:51:49,987 INFO Finished down

In [6]:
df.read_dataset()

<xarray.Dataset>
Dimensions:    (longitude: 97, latitude: 96, time: 720)
Coordinates:
  * longitude  (longitude) float64 -180.0 -176.2 -172.5 ... 172.5 176.2 180.0
  * latitude   (latitude) float64 90.0 88.11 86.21 84.32 ... -86.21 -88.11 -90.0
  * time       (time) datetime64[ns] 1990-01-01 ... 1990-03-31T21:00:00
Data variables:
    XCO2       (time, latitude, longitude) float32 dask.array<chunksize=(248, 96, 97), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    title:        Global atmospheric concentrations inferred by atmospheric i...
    institution:  LSCE (CEA/CNRS/UVSQ) - IPSL
    source:       Poor Man s inversion
    history:      2022-07-19 19:53:47 GMT, PYVAR run by p24cheva
    references:   Chevallier et al. (JGR, 2005, JGR, 2010, ACP, 2019)

In [ ]:
add_var(data, 'carbon_dioxide')
with open("eac4_config.yaml", "w") as file:
    yaml.dump(data, file, sort_keys=False)
data = {}